# Imports

In [1]:
# !pip install espnet
# !pip install espnet_model_zoo

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/asr_hacker')
import espnet
from functions import Speech2TextStreaming
from espnet_model_zoo.downloader import ModelDownloader
import argparse
import numpy as np
import wave

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Download Pretrained Models (ASR Transformer and Language Model)

In [3]:
d=ModelDownloader()
# tag='Emiru Tsunoo/aishell_asr_train_asr_streaming_transformer_raw_zh_char_sp_valid.acc.ave'
tag='D-Keqi/espnet_asr_train_asr_streaming_transformer_raw_en_bpe500_sp_valid.acc.ave'

# print(d.download_and_unpack(tag)['asr_train_config'])
# print(d.download_and_unpack(tag)['lm_train_config'])
# print(d.download_and_unpack(tag)['asr_model_file'])
# print(d.download_and_unpack(tag)['lm_file'])

/usr/local/lib/python3.7/dist-packages/espnet_model_zoo/D-Keqi--espnet_asr_train_asr_streaming_transformer_raw_en_bpe500_sp_valid.acc.ave.main.948310c11d224280984270ae4a3e9792b2fbffce/exp/asr_train_asr_streaming_transformer_raw_en_bpe500_sp/config.yaml
/usr/local/lib/python3.7/dist-packages/espnet_model_zoo/D-Keqi--espnet_asr_train_asr_streaming_transformer_raw_en_bpe500_sp_valid.acc.ave.main.948310c11d224280984270ae4a3e9792b2fbffce/exp/lm_train_lm_en_bpe500/config.yaml
/usr/local/lib/python3.7/dist-packages/espnet_model_zoo/D-Keqi--espnet_asr_train_asr_streaming_transformer_raw_en_bpe500_sp_valid.acc.ave.main.948310c11d224280984270ae4a3e9792b2fbffce/exp/asr_train_asr_streaming_transformer_raw_en_bpe500_sp/valid.acc.ave_10best.pth
/usr/local/lib/python3.7/dist-packages/espnet_model_zoo/D-Keqi--espnet_asr_train_asr_streaming_transformer_raw_en_bpe500_sp_valid.acc.ave.main.948310c11d224280984270ae4a3e9792b2fbffce/exp/lm_train_lm_en_bpe500/19epoch.pth


# Inference

In [4]:
# Initialize speech2text pipeline

speech2text = Speech2TextStreaming(
    **d.download_and_unpack(tag),
    token_type=None,
    bpemodel=None,
    maxlenratio=0.0,
    minlenratio=0.0,
    beam_size=20,
    ctc_weight=0.5,
    lm_weight=0.0,
    penalty=0.0,
    nbest=1,
    device = "cpu",
    disable_repetition_detection=True,
    decoder_text_length_limit=0,
    encoded_feat_length_limit=0
)

In [5]:
prev_lines = 0
def progress_output(text):
    global prev_lines
    lines=['']
    for i in text:
        if len(lines[-1]) > 100:
            lines.append('')
        lines[-1] += i
    for i,line in enumerate(lines):
        if i == prev_lines:
            sys.stderr.write('\n\r')
        else:
            sys.stderr.write('\r\033[B\033[K')
        sys.stderr.write(line)

    prev_lines = len(lines)
    sys.stderr.flush()

In [6]:
def recognize(wavfile):
    with wave.open(wavfile, 'rb') as wavfile:
        ch=wavfile.getnchannels()
        bits=wavfile.getsampwidth()
        rate=wavfile.getframerate()
        nframes=wavfile.getnframes()
        buf = wavfile.readframes(-1)
        data=np.frombuffer(buf, dtype='int16')
    speech = data.astype(np.float16)/32767.0 #32767 is the upper limit of 16-bit binary numbers and is used for the normalization of int to float.
    sim_chunk_length = 640
    if sim_chunk_length > 0:
        for i in range(len(speech)//sim_chunk_length):
            results = speech2text(speech=speech[i*sim_chunk_length:(i+1)*sim_chunk_length], is_final=False)
            if results is not None and len(results) > 0:
                nbests = [text for text, token, token_int, hyp in results]
                text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
                progress_output(nbests[0])
            else:
                progress_output("")
            
        results = speech2text(speech[(i+1)*sim_chunk_length:len(speech)], is_final=True)
    else:
        results = speech2text(speech, is_final=True)
    nbests = [text for text, token, token_int, hyp in results]
    progress_output(nbests[0])


In [7]:
wavfile = '/content/drive/MyDrive/asr_hacker/english_sample.wav'
recognize(wavfile)

/usr/local/lib/python3.7/dist-packages/espnet2/layers/stft.py:166: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  olens = (ilens - self.n_fft) // self.hop_length + 1

/usr/local/lib/python3.7/dist-packages/espnet/nets/batch_beam_search.py:107: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prev_hyp_ids = top_ids // self.n_vocab
he began